- volatile gpu util: GPU的瞬时利用率

## gpu util 优化

- 如果出现 GPU 显存占用很高，模型也在 cuda 上面执行，但训练速度很慢，且 Gpu-Util 很低的情况； 可能原因是 **CPU 的数据加载太慢**，尤其是在多卡服务器中，多个程序同时执行，这个问题会更加严重。
    - 此时应该在 PyTorch 的 DataLoader 中设置 `pin_memory=True`，以及 `num_workers` 参数：

```
from torch.utils.data import DataLoader
train_dataloader = DataLoader(dataset, batch_size, pin_memory=True, num_workers=4)
```

- pin_memory 用于设置是否在 dataloader 返回数据之前将 Tensors 复制到 device/CUDA 中，默认为 False。其作用是**锁页内存**，将数据存储在的固定内存页上，不与硬盘进行内存交换，从而提高将数据从 CPU 传输到 GPU 的效率。
- num_workers 用于设置加载数据的多进程的数量，默认为 0。在 windows 系统下可能只能设置为 0，在 linux 中可以设置为大于 1 的数，具体设置与服务器的 CPU 数量、batch_size 等参数有关。
    - 实际使用中，在 batch_size=32 时，设置 pin_memory=True 和 num_workers=4 的训练速度是设置 pin_memory=False和 num_workers=0 的 40 倍。

In [3]:
import os
os.environ['http_proxy'] = 'http://127.0.0.1:7890'
os.environ['https_proxy'] = 'http://127.0.0.1:7890'

In [9]:
from time import time
import multiprocessing as mp
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

dataset_train = datasets.CIFAR10('./data', train=True, download=True, 
                             transform=transforms.Compose([transforms.ToTensor()]))

dataset_test = datasets.CIFAR10('./data', train=False, download=True, 
                             transform=transforms.Compose([transforms.ToTensor()]))

Files already downloaded and verified
Files already downloaded and verified


In [12]:
results = []
for pin_memory in tqdm([False, True]):
    for batch_size in tqdm([64,  512,  2048]):
        for num_workers in tqdm(range(0, mp.cpu_count(), 16)):  
            train_loader = DataLoader(dataset_train, shuffle=True, 
                                      pin_memory=pin_memory, 
                                      num_workers=num_workers,
                                      batch_size=batch_size,
                                     )
            
            start = time()
            for epoch in range(1, 3):
                for i, data in enumerate(train_loader, 0):
                    pass
            end = time()
            results.append([pin_memory, batch_size, num_workers, end-start])

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/4 [00:00<?, ?it/s]

In [13]:
import pandas as pd
df = pd.DataFrame(results, columns=['pin_memory', 'batch_size', 'num_workers', 'duration'])
df

,pin_memory,batch_size,num_workers,duration
0,False,64,0,7.810005
1,False,64,16,2.141089
2,False,64,32,1.844908
3,False,64,48,1.991111
4,False,512,0,7.491087
5,False,512,16,1.264585
6,False,512,32,1.248658
7,False,512,48,1.439797
8,False,2048,0,7.882908
9,False,2048,16,1.686364
